EM 538: Practical Machine Learning for Enginnering Analytics (Spring 2025)  
Instructor: Fred Livingston (fjliving@ncsu.edu)  
Student: Mike Keating


## Drug Classification Modeling


In [3]:
import pandas as pd
import torch
from sklearn import preprocessing
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

In [5]:
df = pd.read_csv("drug200.csv")
df.head()

,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,F,HIGH,HIGH,25.355,DrugY
1,47,M,LOW,HIGH,13.093,drugC
2,47,M,LOW,HIGH,10.114,drugC
3,28,F,NORMAL,HIGH,7.798,drugX
4,61,F,LOW,HIGH,18.043,DrugY


#### Data Preprocessing

This section is complete for this data set


In [6]:
# Data Encoding
from sklearn.preprocessing import LabelEncoder

data_tobe_encoded = ["Sex", "BP", "Cholesterol", "Drug"]
df_copy = df.copy()
label_encoder = LabelEncoder()
for column in data_tobe_encoded:
    df_copy[column] = label_encoder.fit_transform(df_copy[column])

df_copy.head()


,Age,Sex,BP,Cholesterol,Na_to_K,Drug
0,23,0,0,0,25.355,0
1,47,1,1,0,13.093,3
2,47,1,1,0,10.114,3
3,28,0,2,0,7.798,4
4,61,0,1,0,18.043,0


In [7]:
X = df_copy.drop("Drug", axis=1).values
y = df_copy["Drug"].values

In [8]:
# Data Scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X = scaler.fit_transform(X)

In [9]:
# Data Splitting
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1, stratify=y)

### Torch DataSet and Loader

Task 1. Design a Data Loader for a batch size of 32 with suffling [10/100 pts]


In [36]:
# Dataset
class DrugDataset(Dataset):
    def __init__(self, X, y):
        self.features = torch.tensor(X, dtype=torch.float32)
        self.labels = torch.tensor(y, dtype=torch.int64)

    def __getitem__(self, index):
        return self.features[index], self.labels[index]

    def __len__(self):
        return self.labels.shape[0]

torch.manual_seed(123)

# DataLoader
train_ds = DrugDataset(X_train, y_train)
test_ds = DrugDataset(X_test, y_test)
train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)


#### MLP Model

Task 2: Design a Feed Foward Neural Network that has a test accuracy greater than 90 % [90/100 pts ]


In [66]:
class PyTorchMLP(torch.nn.Module):
    def __init__(self, num_features, num_classes):
        super().__init__()

        self.all_layers = torch.nn.Sequential(
            # input layer
            torch.nn.Linear(num_features, 64),
            torch.nn.ReLU(),
            # output layer
            torch.nn.Linear(64, num_classes),
            torch.nn.Softmax(),
        )

    def forward(self, x):
        z = self.all_layers(x)
        return z


num_features = X.shape[1]
num_classes = len(df_copy["Drug"].unique())

learning_rate = 0.3  # Just selecting a small learning rate for now
model = PyTorchMLP(num_features=num_features, num_classes=num_classes)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  # Stochastic gradient descent
num_epochs = 60  # Number of epochs to train the model

#### Train Model


In [67]:
def train_model(num_epochs, model, optimizer, train_loader):
    for epoch in range(num_epochs):
        model = model.train()
        for batch_idx, (features, labels) in enumerate(train_loader):
            z = model(features)

            loss = F.cross_entropy(z, labels)  # Loss function

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print("Epoch [{}/{}], Loss: {:.4f}".format(epoch + 1, num_epochs, loss.item()))


train_model(num_epochs, model, optimizer, train_loader)

Epoch [1/60], Loss: 1.5560
Epoch [2/60], Loss: 1.5023
Epoch [3/60], Loss: 1.3872
Epoch [4/60], Loss: 1.3104
Epoch [5/60], Loss: 1.4066
Epoch [6/60], Loss: 1.2159
Epoch [7/60], Loss: 1.3266
Epoch [8/60], Loss: 1.1888
Epoch [9/60], Loss: 1.1849
Epoch [10/60], Loss: 1.2933
Epoch [11/60], Loss: 1.1663
Epoch [12/60], Loss: 1.2561
Epoch [13/60], Loss: 1.2292
Epoch [14/60], Loss: 1.2381
Epoch [15/60], Loss: 1.1348
Epoch [16/60], Loss: 1.1522
Epoch [17/60], Loss: 1.1411
Epoch [18/60], Loss: 1.2241
Epoch [19/60], Loss: 1.0485
Epoch [20/60], Loss: 1.1121
Epoch [21/60], Loss: 1.0987
Epoch [22/60], Loss: 1.1354
Epoch [23/60], Loss: 1.1214
Epoch [24/60], Loss: 1.1385
Epoch [25/60], Loss: 1.0330
Epoch [26/60], Loss: 1.1235
Epoch [27/60], Loss: 1.0824
Epoch [28/60], Loss: 1.1453
Epoch [29/60], Loss: 1.0765
Epoch [30/60], Loss: 1.0553
Epoch [31/60], Loss: 1.1340
Epoch [32/60], Loss: 1.0187
Epoch [33/60], Loss: 1.0485
Epoch [34/60], Loss: 1.0577
Epoch [35/60], Loss: 0.9997
Epoch [36/60], Loss: 1.0790
E

In [ ]:
# Adjust lr first, and then epoch until we reach a good accuracy

#### Model Accuracy


In [71]:
def test_model(model, test_loader):
    model = model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for features, labels in test_loader:
            outputs = model(features)
            _, predicted = torch.max(outputs, 1)  # Choose the best class from the output: The class with the best score
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print("Accuracy: {} %".format(100 * correct / total))


test_model(model, test_loader)

Accuracy: 90.0 %


#### Saving & Loading Entire Model


In [ ]:
# TODO Save the model in the following format and submit it
torch.save(model, "hw4_mike_keating.mdl")  # Save the model